In [42]:
# basic imports
import os, random
import pandas as pd
import numpy as np
import datetime as dt
import pandas_ta as ta
from pathlib import Path
import yfinance as yf
import math
import inspect

# import boruta
from boruta import BorutaPy

# import minisom
from minisom import MiniSom

# warnings
import warnings
warnings.filterwarnings('ignore')

# plotting & outputs
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

# sklearn imports
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA

# XGBoost
from xgboost import XGBClassifier

# metrics
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, auc, roc_curve, roc_curve

# import classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

# tensorflow
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator 
from tensorflow.keras.optimizers.schedules import ExponentialDecay

#from tensorflow.keras.optimizers import Adam, RMSprop 
from tensorflow.keras.optimizers.legacy import Adam, RMSprop 
from tensorflow.keras.losses import BinaryCrossentropy 
from tensorflow.keras.metrics import BinaryAccuracy, Accuracy, AUC, Precision, Recall, F1Score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Dropout, Dense, Flatten
from tensorflow.keras.layers import LSTM, BatchNormalization, Bidirectional, GRU

# kerastuner 
import keras_tuner as kt
from keras_tuner import HyperParameters
from keras_tuner.tuners import RandomSearch, BayesianOptimization, Hyperband

# Set display options
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 1000)

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# # Machine info & package version
from watermark import watermark
%load_ext watermark
%watermark -a "Siqi He" -u -d -v -m -iv  

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Author: Siqi He

Last updated: 2024-01-10

Python implementation: CPython
Python version       : 3.8.18
IPython version      : 8.12.2

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.0.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

yfinance   : 0.2.33
numpy      : 1.22.4
pandas_ta  : 0.3.14b0
matplotlib : 3.7.2
pandas     : 1.5.3
tensorflow : 2.13.1
seaborn    : 0.13.0
keras_tuner: 1.3.5



### Set up: Load datasets

In [2]:
# Data preparation: Load scaled dataframes for Train, Validation, Test
    ## NOTE: get raw set for Y - and write new function for different thresholds

# LOAD CSV: raw Y values 
Y_train_df_raw = pd.read_csv("data/Y_train_df_raw.csv",index_col=0)
Y_dev_df_raw = pd.read_csv("data/Y_dev_df_raw.csv",index_col=0)
Y_test_df_raw = pd.read_csv("data/Y_test_df_raw.csv",index_col=0)

# LOAD CSV: Filtered - after removing high-corr
X_train_df_scaled_corr_filtered = pd.read_csv("data/X_train_df_scaled_corr_filtered.csv",index_col=0).to_numpy()
X_dev_df_scaled_corr_filtered = pd.read_csv("data/X_dev_df_scaled_corr_filtered.csv",index_col=0)
X_test_df_scaled_corr_filtered = pd.read_csv("data/X_test_df_scaled_corr_filtered.csv",index_col=0)

# LOAD CSV: Reduced dimension - after Kmeans and SOM
X_train_df_scaled_kmeans_som = pd.read_csv("data/X_train_df_scaled_kmeans_som.csv",index_col=0)
X_dev_df_scaled_kmeans_som = pd.read_csv("data/X_dev_df_scaled_kmeans_som.csv",index_col=0)
X_test_df_scaled_kmeans_som = pd.read_csv("data/X_test_df_scaled_kmeans_som.csv",index_col=0)

# LOAD CSV: Further reduced by XGBoost
X_train_df_scaled_xg = pd.read_csv("data/X_train_df_scaled_xg.csv",index_col=0)
X_dev_df_scaled_xg = pd.read_csv("data/X_dev_df_scaled_xg.csv",index_col=0)
X_test_df_scaled_xg = pd.read_csv("data/X_test_df_scaled_xg.csv",index_col=0)

# view sample feature set
X_train_df_scaled_xg

,WILLR_14,BBP_5_2.0,VCHG_2,STD_63,GAP_3,PCHG_2,CKSPl_10_3_20,STD_3,Mkt-RF,VHF_28,MACDh_12_26_9,VCHG_63,GAP_4,THERMO_20_2_0.5,PVO_12_26_9,...,PPOh_12_26_9,PVOh_12_26_9,GBPUSD=X_Close,GAP_2,SMB,VCHG_1,DMP_14,^FTSE_Close,VTXM_14,dsin,BBB_5_2.0,HML,ER_10,AD,MASSI_9_25
0,0.076734,-0.390621,-0.170862,-0.090478,-0.273369,-1.031079,1.623661,0.072627,0.486486,0.613907,1.290233,0.219413,-0.221454,0.430629,-0.479601,...,0.374750,0.140792,0.228313,-0.677519,-0.020408,-0.139776,2.283319,0.259824,-0.277996,0.433884,0.177006,0.043478,0.482508,1.509226,1.102748
1,0.059680,-0.485234,-0.415944,-0.115661,-0.728628,-0.344201,1.639169,-0.198157,-0.054054,0.713555,0.971918,-0.522727,-0.379045,0.897359,-0.624051,...,0.322398,-0.099869,0.608384,-0.179615,0.489796,-0.279575,2.001703,-0.053300,-0.330060,-0.433884,0.104483,-0.282609,-0.049717,1.512624,1.025024
2,0.092805,-0.246218,-0.609040,-0.115434,-0.283141,0.025803,1.657413,-0.598468,0.032432,0.696304,0.731593,-0.448552,-0.713536,-0.595914,-0.868236,...,0.385694,-0.447878,-0.254330,-0.437854,0.244898,-0.403336,1.838463,-0.124135,-0.473537,-0.974928,0.006146,1.847826,-0.135956,1.515244,0.912759
3,0.028021,-0.520160,-0.399661,-0.115259,0.302931,-0.098843,1.670525,-0.743685,0.129730,0.366477,0.461385,-0.427586,0.295895,-0.726579,-1.068856,...,0.239335,-0.658370,0.124259,0.603449,0.714286,0.032720,1.646422,0.421903,-0.796591,0.974928,-0.730399,0.043478,0.071471,1.503808,0.686135
4,-0.045053,-0.826751,0.806589,-0.115410,0.839802,-0.375475,1.682877,-0.679642,-0.378378,0.570205,0.178100,-0.246017,0.527492,0.020029,-1.005650,...,0.136767,-0.415475,-0.275278,1.214832,0.510204,0.952643,1.410299,0.054196,-0.641540,0.433884,-0.762788,0.695652,-0.273531,1.499302,0.440891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1426,0.687365,-0.081214,-0.181734,0.031710,-0.428149,-0.198224,0.852888,0.038799,0.281081,0.550486,-0.382766,0.181716,-0.936902,0.955415,-0.233002,...,-0.359198,0.308251,-0.075513,-0.570349,0.183673,-0.013083,1.163784,0.289411,-0.082615,-0.433884,-0.514799,-0.956522,-0.797456,-1.122057,0.486153
1427,-0.807432,-0.721870,0.345706,0.043626,-0.428012,-0.262307,0.852888,0.225531,-0.086486,0.515266,-0.627028,0.246060,-0.330564,0.540587,0.064174,...,-0.507885,0.722566,-0.402738,0.620943,0.061224,0.492244,0.962135,-0.194257,-0.138207,-0.974928,-0.291017,-0.260870,-0.109200,-1.173370,0.341962
1428,-0.661451,-0.562555,-0.507586,-0.219410,0.604362,-0.732682,0.852888,0.091204,0.021622,0.790326,-0.811064,-0.819929,-0.270979,-0.198148,-0.122326,...,-0.709189,0.299998,-0.274547,0.129799,0.428571,-0.774613,0.774889,0.382888,-0.101190,0.781831,-0.154240,-0.130435,0.716448,-1.193657,0.315068
1429,-0.422076,-0.098798,-0.765551,-0.230603,0.281931,0.092260,0.859820,-0.049326,0.118919,0.815707,-0.836908,0.028405,0.629279,0.693120,-0.296110,...,-0.861694,-0.018233,-0.685270,0.311243,-0.326531,-0.040872,0.601019,0.022318,0.142316,0.974928,-0.422076,-0.500000,0.211338,-1.175907,0.272505


### Set up: Class weights and sequential data

In [3]:
# set class weights

def cwts(dfs):
    '''
    Calculates class weights based on target value counts in a numpy array or pandas dataframe of target variable Y.
    '''
    c0, c1 = np.bincount(dfs)
    w0=(1/c0)*(len(dfs))/2 
    w1=(1/c1)*(len(dfs))/2 
    
    return {0: w0, 1: w1}

# time series generator to transform data for LSTM input

def time_series_generator(X, Y, seqlen):
    '''
    Calls the TimeSeriesGenerator module in Tensorflow.preprocessing.sequence.
    For a given pair of X, Y and lookback period
    '''
    return TimeseriesGenerator(X, Y, length=seqlen)

# turn Y raw values into direction labels

y_train = np.where(Y_train_df_raw['label']>0.001, 1, 0)
y_dev = np.where(Y_dev_df_raw['label']>0.001, 1, 0)
y_test = np.where(Y_test_df_raw['label']>0.001, 1, 0)

# create classweights

class_weight = cwts(y_train)

# create sequential data

g_train_corr_filtered = time_series_generator(X_train_df_scaled_corr_filtered, y_train, seqlen=21)
g_dev_corr_filtered = time_series_generator(X_dev_df_scaled_corr_filtered, y_dev, seqlen=21)
g_test_corr_filtered = time_series_generator(X_test_df_scaled_corr_filtered, y_test, seqlen=21)

g_train_kmeans_som = time_series_generator(X_train_df_scaled_kmeans_som, y_train, seqlen=21)
g_dev_kmeans_som = time_series_generator(X_dev_df_scaled_kmeans_som, y_dev, seqlen=21)
g_test_kmeans_som = time_series_generator(X_test_df_scaled_kmeans_som, y_test, seqlen=21)

g_train_xg = time_series_generator(X_train_df_scaled_xg, y_train, seqlen=21)
g_dev_xg = time_series_generator(X_dev_df_scaled_xg, y_dev, seqlen=21)
g_test_xg = time_series_generator(X_test_df_scaled_xg, y_test, seqlen=21)

# Visualise sequential data shape

print("Sequential data after filtering high-correlation: " )
for i in range(len(g_train_corr_filtered)):
    a, b = g_train_corr_filtered[i]
    print(a.shape, b.shape)
print()
print("Sequential data from K-means clustering and SOM selection: " )
for i in range(len(g_train_kmeans_som)):
    a, b = g_train_kmeans_som[i]
    print(a.shape, b.shape)
print()
print("Sequential data from XGBoost selection: " )
for i in range(len(g_train_xg)):
    a, b = g_train_xg[i]
    print(a.shape, b.shape)
    
# Other global parameters

num_features_kmeans_som = X_train_df_scaled_kmeans_som.shape[1]                       # number of features selected by K-means and SOM
num_features_corr_filtered = X_train_df_scaled_corr_filtered.shape[1]                 # number of features remaining after filtering high correlation
num_features_xg = X_train_df_scaled_xg.shape[1]                                       # number of features selectee after XGBoost
S2 = 40         # Number of units in the last dense layer before LSTM units
S1 = 80         # Number of units in the second-to-last dense layer before LSTM units
LR = 0.005      # Learning rate
seqlen = 21     # lookback period
BATCH = 64     # batch size
EPOCHS = 50    # number of epochs
PATIENCE = 10  # patience
logdir = f"./tensorboard/LSTM/initial_run/"  # tensorboard root directory

Sequential data after filtering high-correlation: 
(128, 21, 199) (128,)
(128, 21, 199) (128,)
(128, 21, 199) (128,)
(128, 21, 199) (128,)
(128, 21, 199) (128,)
(128, 21, 199) (128,)
(128, 21, 199) (128,)
(128, 21, 199) (128,)
(128, 21, 199) (128,)
(128, 21, 199) (128,)
(128, 21, 199) (128,)
(2, 21, 199) (2,)

Sequential data from K-means clustering and SOM selection: 
(128, 21, 96) (128,)
(128, 21, 96) (128,)
(128, 21, 96) (128,)
(128, 21, 96) (128,)
(128, 21, 96) (128,)
(128, 21, 96) (128,)
(128, 21, 96) (128,)
(128, 21, 96) (128,)
(128, 21, 96) (128,)
(128, 21, 96) (128,)
(128, 21, 96) (128,)
(2, 21, 96) (2,)

Sequential data from XGBoost selection: 
(128, 21, 37) (128,)
(128, 21, 37) (128,)
(128, 21, 37) (128,)
(128, 21, 37) (128,)
(128, 21, 37) (128,)
(128, 21, 37) (128,)
(128, 21, 37) (128,)
(128, 21, 37) (128,)
(128, 21, 37) (128,)
(128, 21, 37) (128,)
(128, 21, 37) (128,)
(2, 21, 37) (2,)


### LSTM Model architectures

In [4]:

def LSTM_model_arch_1(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    - 0 dense layers before LSTM layers
    - 1 LSTM layer, with drop-out
    - 0 dense layer before output
    
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard location
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, features), activation = 'elu', return_sequences=False, name='LSTM1'))

    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)
    
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              verbose=1, 
              callbacks=my_callbacks, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_10(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    
    - 2 dense layer at start
    - 1 LSTM layer, with drop-out layer
    - 0 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")

    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features),)) 
    model.add(Dense(units=S2, name='Dense_start_2')) 
    
    # first LSTM layer
    model.add(LSTM(units=hu, input_shape=(lookback, S2), activation = 'elu', return_sequences=False, name='LSTM1'))
    
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)
    
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    
    model.build(input_shape=(None, lookback, features))
    
    # print model summary
    model.summary()

    

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              verbose=1, 
              callbacks=my_callbacks, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_11(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    
    - 3 dense layer at start
    - 1 LSTM layer, with drop-out layer
    - 0 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features))) 
    model.add(Dense(units=S1, name='Dense_start_2')) 
    model.add(Dense(units=S2, name='Dense_start_3')) 
    
    # first LSTM layer
    model.add(LSTM(units=hu, input_shape=(lookback, S2), activation = 'elu', return_sequences=False, name='LSTM1'))
    
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)
    
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              verbose=1, 
              callbacks=my_callbacks, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_2(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    - 0 Dense layer before LSTM layers
    - 1 LSTM layer, with drop-out layer
    - One dense layer
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # first LSTM layer
    model.add(LSTM(units=hu, input_shape=(lookback, features), activation = 'elu', return_sequences=False, name='LSTM1'))
    
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_1'))

    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              verbose=1, 
              callbacks=my_callbacks, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_12(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    - 2 dense layer before LSTM
    - 1 LSTM layer, with drop-out layer
    - 1 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features))) 
    model.add(Dense(units=S2, name='Dense_start_2')) 

    # first LSTM layer
    model.add(LSTM(units=hu, input_shape=(lookback, S2), activation = 'elu', return_sequences=False, name='LSTM1'))
    
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_1'))

    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              verbose=1, 
              callbacks=my_callbacks, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_13(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    - 3 dense layer before LSTM
    - 1 LSTM layer, with drop-out layer
    - 1 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features))) 
    model.add(Dense(units=S1, name='Dense_start_2')) 
    model.add(Dense(units=S2, name='Dense_start_3')) 

    # first LSTM layer
    model.add(LSTM(units=hu, input_shape=(lookback, S2), activation = 'elu', return_sequences=False, name='LSTM1'))
    
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_1'))

    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              verbose=1, 
              callbacks=my_callbacks, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_3(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    
    - 0 dense layer before LSTM
    - 1 LSTM layer, with drop-out layer
    - 2 dense layers before output
    
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, features), activation = 'elu', return_sequences=False, name='LSTM1'))
    
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # first dense layer
    model.add(Dense(units=int(hu*3/4), activation='elu', name='Dense_end_1'))

    # second dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_2'))

    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))   

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_14(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    
    - 2 dense layer before LSTM
    - 1 LSTM layer, with drop-out layer
    - 2 dense layers before output
    
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features))) 
    model.add(Dense(units=S2, name='Dense_start_2')) 

    # first LSTM layer
    model.add(LSTM(units=hu, input_shape=(lookback, S2), activation = 'elu', return_sequences=False, name='LSTM1'))
    
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # first dense layer
    model.add(Dense(units=int(hu*3/4), activation='elu', name='Dense_end_1'))

    # second dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_2'))

    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))   

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_15(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    
    - 3 dense layer before LSTM
    - 1 LSTM layer, with drop-out layer
    - 2 dense layers before output
    
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features))) 
    model.add(Dense(units=S1, name='Dense_start_2')) 
    model.add(Dense(units=S2, name='Dense_start_3')) 

    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, S2), activation = 'elu', return_sequences=False, name='LSTM1'))
    
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # first dense layer
    model.add(Dense(units=int(hu*3/4), activation='elu', name='Dense_end_1'))

    # second dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_2'))

    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))   

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,epochs=EPOCHS, batch_size = BATCH, verbose=1, shuffle=False,class_weight=class_weights,validation_data=g_valid)

    return model, history

def LSTM_model_arch_4(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Baseline model architecture: 
    
    - 0 dense layer before LSTM
    - 2 LSTM layers, with drop-out layers
    - 0 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, features), activation = 'elu', return_sequences=True, name='LSTM1'))
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM2'))

    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              verbose=1, 
              callbacks=my_callbacks, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_16(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    
    - 2 dense layer before LSTM
    - 2 LSTM layers, with drop-out layers
    - 0 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features))) 
    model.add(Dense(units=S2, name='Dense_start_2')) 

    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, S2), activation = 'elu', return_sequences=True, name='LSTM1'))
    
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM2'))

    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              verbose=1, 
              callbacks=my_callbacks, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_17(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    
    - 3 dense layer before LSTM
    - 2 LSTM layers, with drop-out layers
    - 0 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features))) 
    model.add(Dense(units=S1, name='Dense_start_2')) 
    model.add(Dense(units=S2, name='Dense_start_3')) 

    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, S2), activation = 'elu', return_sequences=True, name='LSTM1'))
    
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM2'))

    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              verbose=1, 
              callbacks=my_callbacks, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_5(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    - 0 dense layers before LSTM layers
    - 2 LSTM layers, with drop-out layers
    - 1 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, features), activation = 'elu', return_sequences=True, name='LSTM1'))
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'relu', return_sequences=False, name='LSTM2'))

    # first dense layer
    model.add(Dense(units=int(hu/2), activation='relu', name='Dense1'))
    
    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              callbacks=my_callbacks, 
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_18(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 

    - 2 dense layers before LSTM
    - 2 LSTM layers, with drop-out layers
    - 1 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features))) 
    model.add(Dense(units=S2, name='Dense_start_2')) 

    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, S2), activation = 'elu', return_sequences=True, name='LSTM1'))
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM2'))

    # first dense layer
    model.add(Dense(units=int(hu*3/4), activation='elu', name='Dense_end_1'))
    
    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              callbacks=my_callbacks, 
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_19(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 

    - 3 dense layers before LSTM
    - 2 LSTM layers, with drop-out layers
    - 1 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features))) 
    model.add(Dense(units=S1, name='Dense_start_2')) 
    model.add(Dense(units=S2, name='Dense_start_3')) 

    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, S2), activation = 'elu', return_sequences=True, name='LSTM1'))
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM2'))

    # first dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_1'))
    
    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,epochs=EPOCHS, batch_size = BATCH, callbacks=my_callbacks, verbose=1, shuffle=False, class_weight=class_weights, validation_data=g_valid)

    return model, history

def LSTM_model_arch_6(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    - 0 dense layers before LSTM layers
    - 2 LSTM layers, with drop-out layers
    - 2 dense layer before output layer
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, features), activation = 'elu', return_sequences=True, name='LSTM1'))
    
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM2'))

    # first dense layer
    model.add(Dense(units=int(hu*3/4), activation='elu', name='Dense_end_1'))
    
    # second dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_2'))
    
    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              callbacks=my_callbacks, 
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_20(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    - 2 dense layers before LSTM layers
    - 2 LSTM layers, with drop-out layers
    - 2 dense layer before output layer
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1',input_shape=(lookback, features))) 
    model.add(Dense(units=S2, name='Dense_start_2')) 

    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, S2), activation = 'elu', return_sequences=True, name='LSTM1'))
    
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM2'))

    # first dense layer
    model.add(Dense(units=int(hu*3/4), activation='elu', name='Dense_end_1'))
    
    # second dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_2'))
    
    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              callbacks=my_callbacks, 
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_21(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    - 3 dense layers before LSTM layers
    - 2 LSTM layers, with drop-out layers
    - 2 dense layer before output layer
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features))) 
    model.add(Dense(units=S1, name='Dense_start_2')) 
    model.add(Dense(units=S2, name='Dense_start_3')) 

    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, S2), activation = 'elu', return_sequences=True, name='LSTM1'))
    
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM2'))

    # first dense layer
    model.add(Dense(units=int(hu*3/4), activation='elu', name='Dense_end_1'))
    
    # second dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_2'))
    
    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              callbacks=my_callbacks, 
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_7(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Baseline model architecture: 

    - 0 dense layer before LSTM layers
    - 3 LSTM layers, with drop-out layers
    - 0 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, features), activation = 'elu', return_sequences=True, name='LSTM1'))
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=True, name='LSTM2'))
    # second dropout layer
    model.add(Dropout(0.4, name='Dropout2'))

    # third LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM3'))

    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              callbacks=my_callbacks, 
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_22(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Baseline model architecture: 

    - 2 dense layer before LSTM layers
    - 3 LSTM layers, with drop-out layers
    - 0 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features))) 
    model.add(Dense(units=S2, name='Dense_start_2')) 
    
    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, S2), activation = 'elu', return_sequences=True, name='LSTM1'))
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=True, name='LSTM2'))
    # second dropout layer
    model.add(Dropout(0.4, name='Dropout2'))

    # third LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM3'))

    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              callbacks=my_callbacks, 
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_23(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 

    - 3 dense layer before LSTM layers
    - 3 LSTM layers, with drop-out layers
    - 0 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features))) 
    model.add(Dense(units=S1, name='Dense_start_2')) 
    model.add(Dense(units=S2, name='Dense_start_3')) 
    
    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, S2), activation = 'elu', return_sequences=True, name='LSTM1'))
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=True, name='LSTM2'))
    # second dropout layer
    model.add(Dropout(0.4, name='Dropout2'))

    # third LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM3'))

    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              callbacks=my_callbacks, 
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_8(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    
    - 0 dense layers before LSTM layers
    - 3 LSTM layers, with drop-out layers
    - 1 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, features), activation = 'elu', return_sequences=True, name='LSTM1'))
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=True, name='LSTM2'))
    # second dropout layer
    model.add(Dropout(0.4, name='Dropout2'))

    # third LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM3'))
    
    # first dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_1'))
    
    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              callbacks=my_callbacks, 
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_24(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    
    - 2 dense layers before LSTM layers
    - 3 LSTM layers, with drop-out layers
    - 1 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/model_arch_24_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")

    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features))) 
    model.add(Dense(units=S2, name='Dense_start_2')) 

    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, S2), activation = 'elu', return_sequences=True, name='LSTM1'))
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=True, name='LSTM2'))
    # second dropout layer
    model.add(Dropout(0.4, name='Dropout2'))

    # third LSTM layer
    model.add(LSTM(units=int(hu*3/4), activation = 'elu', return_sequences=False, name='LSTM3'))
    
    # first dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_1'))
    
    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True), TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              callbacks=my_callbacks, 
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_25(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    
    - 3 dense layers before LSTM layers
    - 3 LSTM layers, with drop-out layers
    - 1 dense layer before output
    '''
    
    tf.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features))) 
    model.add(Dense(units=S1, name='Dense_start_2'))
    model.add(Dense(units=S2, name='Dense_start_3')) 

    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, S2), activation = 'elu', return_sequences=True, name='LSTM1'))
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=True, name='LSTM2'))
    # second dropout layer
    model.add(Dropout(0.4, name='Dropout2'))

    # third LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM3'))
    
    # first dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_1'))
    
    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              callbacks=my_callbacks, 
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_9(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    
    - 0 dense layers before LSTM layers
    - 3 LSTM layers, with drop-out layers for the first LSTM layer
    - 2 dense layers before output
    '''
    
    tf.keras.backend.clear_session() 
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, features), activation = 'elu', return_sequences=True, name='LSTM1'))
    
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=True, name='LSTM2'))
    
    # second dropout layer
    model.add(Dropout(0.4, name='Dropout2'))

    # third LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM3'))
    
    # first dense layer
    model.add(Dense(units=int(hu*3/4), activation='elu', name='Dense_end_1'))

    # second dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_2'))
    
    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    model.build(input_shape=(None, lookback, features))

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              callbacks=my_callbacks, 
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_26(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    
    - 2 dense layers before LSTM layers
    - 3 LSTM layers, with drop-out layers for the first LSTM layer
    - 2 dense layers before output
    '''
    
    tf.keras.backend.clear_session() 
    
    # instantiate the model
    model = Sequential()
    
    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features))) 
    model.add(Dense(units=S2, name='Dense_start_2')) 

    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, S2), activation = 'elu', return_sequences=True, name='LSTM1'))
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=True, name='LSTM2'))
    
    # second dropout layer
    model.add(Dropout(0.4, name='Dropout2'))

    # third LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM3'))
    
    # first dense layer
    model.add(Dense(units=int(hu*3/4), activation='elu', name='Dense_end_1'))

    # second dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_2'))
    
    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              callbacks=my_callbacks, 
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history

def LSTM_model_arch_27(g_train, g_valid, features, lookback, class_weights, hu=128, data='high_corr'):
    '''
    Model architecture: 
    
    - 3 dense layers before LSTM layers
    - 3 LSTM layers, with drop-out layers for the first LSTM layer
    - 2 dense layers before output
    '''
    
    tf.keras.backend.clear_session() 
    
    # instantiate the model
    model = Sequential()

    # tensorboard path
    datetime = dt.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    logdir = f"./tensorboard/LSTM/initial_run/{data}/{inspect.currentframe().f_code.co_name}_{datetime}"
    print(f"Now fitting model: {inspect.currentframe().f_code.co_name}")
    
    # dense layer
    model.add(Dense(units=features, name='Dense_start_1', input_shape=(lookback, features),)) 
    model.add(Dense(units=S1, name='Dense_start_2')) 
    model.add(Dense(units=S2, name='Dense_start_3')) 

    # first LSTM layer
    model.add(LSTM(units=hu*2, input_shape=(lookback, S2), activation = 'elu', return_sequences=True, name='LSTM1'))
    # first dropout layer
    model.add(Dropout(0.4, name='Dropout1'))

    # second LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=True, name='LSTM2'))
    
    # second dropout layer
    model.add(Dropout(0.4, name='Dropout2'))

    # third LSTM layer
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM3'))
    
    # first dense layer
    model.add(Dense(units=int(hu*3/4), activation='elu', name='Dense_end_1'))

    # second dense layer
    model.add(Dense(units=int(hu/2), activation='elu', name='Dense_end_2'))
    
    # output layer
    model.add(Dense(units=1, activation='sigmoid', name='Output'))

    # optimizer
    opt = Adam(learning_rate=LR, epsilon=1e-08)

    # callback
    my_callbacks = [
        EarlyStopping(patience=PATIENCE, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True),TensorBoard(log_dir=logdir)
    ]

    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy',
                           Precision(),
                           Recall()])
    # print model summary
    model.summary()

    # fit model
    history = model.fit(g_train,
              epochs=EPOCHS, 
                        batch_size = BATCH,
              callbacks=my_callbacks, 
              verbose=1, 
              shuffle=False,
              class_weight=class_weights,
              validation_data=g_valid)

    return model, history


### Baseline model: Fit to training data

In [51]:
# Create empty dataframe to store results
LSTM_baseline_model_som_kmeans_data =  pd.DataFrame(columns=['Dataset','Model_Architecture', 'Loss', 'Accuracy', 'Precision', 'Recall', 
                                            'Validation_Loss', 
                                            'Validation_Accuracy', 'Validation_Precision', 'Validation_Recall'])

# Fit baseline model
baseline_model_arch, fit_history = LSTM_model_arch_1(g_train_kmeans_som, g_dev_kmeans_som, features=num_features_kmeans_som, lookback=seqlen, class_weights=class_weight, hu=10, data="baseline_trial")
# Get baseline model results on development set
LSTM_baseline_model_som_kmeans_data = LSTM_baseline_model_som_kmeans_data.append({
    'Dataset': "SOM K-means dimentionally reduced ",
    'Model_Architecture': LSTM_model_arch_1.__name__,
    'Loss': fit_history.history['loss'][-1], 
    'Accuracy': fit_history.history['accuracy'][-1], 
    'Precision': fit_history.history['precision'][-1], 
    'Recall': fit_history.history['recall'][-1], 
    'Validation_Loss': fit_history.history['val_loss'][-1], 
    'Validation_Accuracy': fit_history.history['val_accuracy'][-1], 
    'Validation_Precision': fit_history.history['val_precision'][-1], 
    'Validation_Recall': fit_history.history['val_recall'][-1]
},ignore_index=True)

# View baseline model results
LSTM_baseline_model_som_kmeans_data

Now fitting model: LSTM_model_arch_1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 LSTM1 (LSTM)                (None, 20)                9360      
                                                                 
 Dropout1 (Dropout)          (None, 20)                0         
                                                                 
 Output (Dense)              (None, 1)                 21        
                                                                 
Total params: 9381 (36.64 KB)
Trainable params: 9381 (36.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
12/12 [==============================] - 1s 21ms/step - loss: 0.7922 - accuracy: 0.4830 - precision: 0.4475 - recall: 0.5550 - val_loss: 0.7385 - val_accuracy: 0.4934 - val_precision: 0.4706 - val_recall: 0.7583
Epoch 2/50
12/12 [========

,Dataset,Model_Architecture,Loss,Accuracy,Precision,Recall,Validation_Loss,Validation_Accuracy,Validation_Precision,Validation_Recall
0,High-corr filtered,LSTM_model_arch_1,0.605165,0.660284,0.609211,0.717829,0.791142,0.517544,0.481481,0.554502


### Initial runs: Architecture shorlisting

In [7]:
# create dataframe for global best model results
LSTM_global_results_som_kmeans_data = pd.DataFrame(columns=['Dataset','Model_Architecture', 'Loss', 'Accuracy', 'Precision', 'Recall', 
                                            'Validation_Loss', 
                                            'Validation_Accuracy', 'Validation_Precision', 'Validation_Recall'])

LSTM_global_results_filtered_data = pd.DataFrame(columns=['Dataset','Model_Architecture', 'Loss', 'Accuracy', 'Precision', 'Recall', 
                                            'Validation_Loss', 
                                            'Validation_Accuracy', 'Validation_Precision', 'Validation_Recall'])
LSTM_global_results_xg_data = pd.DataFrame(columns=['Dataset','Model_Architecture', 'Loss', 'Accuracy', 'Precision', 'Recall', 
                                            'Validation_Loss', 
                                            'Validation_Accuracy', 'Validation_Precision', 'Validation_Recall'])
# get a list of all model architectures
LSTM_models = [ \
              LSTM_model_arch_1, LSTM_model_arch_2, LSTM_model_arch_3, \
              LSTM_model_arch_4, LSTM_model_arch_5, LSTM_model_arch_6, \
              LSTM_model_arch_7, LSTM_model_arch_8, LSTM_model_arch_9, \
              LSTM_model_arch_10, LSTM_model_arch_11, LSTM_model_arch_12, \
              LSTM_model_arch_13, LSTM_model_arch_14, LSTM_model_arch_15, \
              LSTM_model_arch_16, LSTM_model_arch_17, LSTM_model_arch_18, \
              LSTM_model_arch_19, LSTM_model_arch_20, LSTM_model_arch_21, \
              LSTM_model_arch_22, LSTM_model_arch_23, LSTM_model_arch_24, \
              LSTM_model_arch_25, LSTM_model_arch_26, LSTM_model_arch_27 \
              ]
# Train each model architecture and append local results to global table
for index, model in enumerate(LSTM_models):
    if index in range(10, 28):
        model_arch, fit_history = model(g_train_corr_filtered, g_dev_corr_filtered, features=num_features_corr_filtered, lookback=seqlen, class_weights=class_weight, hu=10, data="hc_trialrun_jan9_7")
        LSTM_global_results_filtered_data = LSTM_global_results_filtered_data.append({
            'Dataset': "High-corr filtered",
            'Model_Architecture': model.__name__,
            'Loss': fit_history.history['loss'][-1], 
            'Accuracy': fit_history.history['accuracy'][-1], 
            'Precision': fit_history.history['precision'][-1], 
            'Recall': fit_history.history['recall'][-1], 
            'Validation_Loss': fit_history.history['val_loss'][-1], 
            'Validation_Accuracy': fit_history.history['val_accuracy'][-1], 
            'Validation_Precision': fit_history.history['val_precision'][-1], 
            'Validation_Recall': fit_history.history['val_recall'][-1]
        },ignore_index=True)
        
        model_arch, fit_history = model(g_train_kmeans_som, g_dev_kmeans_som, features=num_features_kmeans_som, lookback=seqlen, class_weights=class_weight, hu=10, data="sk_trialrun_jan9_6")
        LSTM_global_results_som_kmeans_data = LSTM_global_results_som_kmeans_data.append({
            'Dataset': "SOM K-means dimentionally reduced",
            'Model_Architecture': model.__name__,
            'Loss': fit_history.history['loss'][-1], 
            'Accuracy': fit_history.history['accuracy'][-1], 
            'Precision': fit_history.history['precision'][-1], 
            'Recall': fit_history.history['recall'][-1], 
            'Validation_Loss': fit_history.history['val_loss'][-1], 
            'Validation_Accuracy': fit_history.history['val_accuracy'][-1], 
            'Validation_Precision': fit_history.history['val_precision'][-1], 
            'Validation_Recall': fit_history.history['val_recall'][-1]
        },ignore_index=True)
    
    if index in range(0, 10):
        model_arch, fit_history = model(g_train_kmeans_som, g_dev_kmeans_som, features=num_features_kmeans_som, lookback=seqlen, class_weights=class_weight, hu=10, data="sk_trialrun_jan9_7")
        LSTM_global_results_som_kmeans_data = LSTM_global_results_som_kmeans_data.append({
            'Dataset': "SOM K-means dimentionally reduced",
            'Model_Architecture': model.__name__,
            'Loss': fit_history.history['loss'][-1], 
            'Accuracy': fit_history.history['accuracy'][-1], 
            'Precision': fit_history.history['precision'][-1], 
            'Recall': fit_history.history['recall'][-1], 
            'Validation_Loss': fit_history.history['val_loss'][-1], 
            'Validation_Accuracy': fit_history.history['val_accuracy'][-1], 
            'Validation_Precision': fit_history.history['val_precision'][-1], 
            'Validation_Recall': fit_history.history['val_recall'][-1]
        },ignore_index=True)

        model_arch, fit_history = model(g_train_xg, g_dev_xg, features=num_features_xg, lookback=seqlen, class_weights=class_weight, hu=10, data="xg_trialrun_jan9_7")
        LSTM_global_results_xg_data = LSTM_global_results_xg_data.append({
            'Dataset': "XGBoost",
            'Model_Architecture': model.__name__,
            'Loss': fit_history.history['loss'][-1], 
            'Accuracy': fit_history.history['accuracy'][-1], 
            'Precision': fit_history.history['precision'][-1], 
            'Recall': fit_history.history['recall'][-1], 
            'Validation_Loss': fit_history.history['val_loss'][-1], 
            'Validation_Accuracy': fit_history.history['val_accuracy'][-1], 
            'Validation_Precision': fit_history.history['val_precision'][-1], 
            'Validation_Recall': fit_history.history['val_recall'][-1]
        },ignore_index=True)

        
# # view global results dataframe
LSTM_global_results_filtered_data

Now fitting model: LSTM_model_arch_1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 LSTM1 (LSTM)                (None, 20)                9360      
                                                                 
 Dropout1 (Dropout)          (None, 20)                0         
                                                                 
 Output (Dense)              (None, 1)                 21        
                                                                 
Total params: 9381 (36.64 KB)
Trainable params: 9381 (36.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
12/12 [==============================] - 1s 21ms/step - loss: 0.7626 - accuracy: 0.4986 - precision: 0.4574 - recall: 0.5163 - val_loss: 0.7049 - val_accuracy: 0.5219 - val_precision: 0.4685 - val_recall: 0.2464
Epoch 2/50
12/12 [========

,Dataset,Model_Architecture,Loss,Accuracy,Precision,Recall,Validation_Loss,Validation_Accuracy,Validation_Precision,Validation_Recall
0,High-corr filtered,LSTM_model_arch_11,0.685495,0.566667,0.527778,0.500775,0.695222,0.504386,0.461929,0.431280
1,High-corr filtered,LSTM_model_arch_12,0.621878,0.643972,0.667447,0.441860,0.680536,0.574561,0.545946,0.478673
2,High-corr filtered,LSTM_model_arch_13,0.685344,0.567376,0.597765,0.165891,0.705010,0.543860,0.527273,0.137441
3,High-corr filtered,LSTM_model_arch_14,0.615444,0.653901,0.689157,0.443411,0.773789,0.543860,0.505415,0.663507
4,High-corr filtered,LSTM_model_arch_15,0.695336,0.542553,0.000000,0.000000,0.689996,0.537281,0.000000,0.000000
5,High-corr filtered,LSTM_model_arch_16,0.679260,0.578723,0.545617,0.472868,0.691410,0.567982,0.570000,0.270142
6,High-corr filtered,LSTM_model_arch_17,0.696753,0.539007,0.333333,0.007752,0.691212,0.537281,0.000000,0.000000
7,High-corr filtered,LSTM_model_arch_18,0.694631,0.536170,0.493506,0.530233,0.712441,0.497807,0.452632,0.407583
8,High-corr filtered,LSTM_model_arch_19,0.702665,0.536170,0.480519,0.172093,0.690207,0.535088,0.400000,0.009479
9,High-corr filtered,LSTM_model_arch_20,0.696449,0.572340,0.552239,0.344186,0.717235,0.519737,0.463636,0.241706


### Initial run: View results

In [15]:
# Rank model architecture results in ascending order of Validation Loss
best_models_filtered_data = LSTM_global_results_filtered_data.sort_values(by='Validation_Loss', ascending=True)
best_models_som_kmeans_data = LSTM_global_results_som_kmeans_data.sort_values(by='Validation_Loss', ascending=True)
best_models_xg_data = LSTM_global_results_xg_data.sort_values(by='Validation_Loss', ascending=True)

# view model result dataset for som and kmeans reduced data
best_models_som_kmeans_data

,Dataset,Model_Architecture,Loss,Accuracy,Precision,Recall,Validation_Loss,Validation_Accuracy,Validation_Precision,Validation_Recall
23,SOM K-means dimentionally reduced,LSTM_model_arch_24,0.699207,0.536879,0.445946,0.051163,0.692527,0.519737,0.466102,0.260664
20,SOM K-means dimentionally reduced,LSTM_model_arch_21,0.697844,0.530496,0.478372,0.291473,0.694225,0.513158,0.465839,0.355450
25,SOM K-means dimentionally reduced,LSTM_model_arch_26,0.697293,0.501418,0.444231,0.358140,0.694361,0.500000,0.469091,0.611374
22,SOM K-means dimentionally reduced,LSTM_model_arch_23,0.716011,0.553901,0.527778,0.235659,0.694394,0.519737,0.484252,0.582938
26,SOM K-means dimentionally reduced,LSTM_model_arch_27,0.712774,0.497872,0.393939,0.181395,0.695358,0.563596,0.581081,0.203791
19,SOM K-means dimentionally reduced,LSTM_model_arch_20,0.687325,0.549645,0.506887,0.570543,0.697345,0.524123,0.487179,0.540284
16,SOM K-means dimentionally reduced,LSTM_model_arch_17,0.699650,0.509929,0.471178,0.582946,0.699502,0.495614,0.425197,0.255924
21,SOM K-means dimentionally reduced,LSTM_model_arch_22,0.698798,0.545390,0.505376,0.291473,0.702117,0.489035,0.454545,0.521327
4,SOM K-means dimentionally reduced,LSTM_model_arch_5,0.667673,0.591489,0.561279,0.489922,0.710798,0.519737,0.468750,0.284360
18,SOM K-means dimentionally reduced,LSTM_model_arch_19,0.675467,0.565957,0.517205,0.768992,0.711360,0.508772,0.479624,0.725118


In [64]:
# view model result dataset for high correlation filtered
best_models_filtered_data

,Dataset,Model_Architecture,Loss,Accuracy,Precision,Recall,Validation_Loss,Validation_Accuracy,Validation_Precision,Validation_Recall
1,High-corr filtered,LSTM_model_arch_12,0.621878,0.643972,0.667447,0.441860,0.680536,0.574561,0.545946,0.478673
4,High-corr filtered,LSTM_model_arch_15,0.695336,0.542553,0.000000,0.000000,0.689996,0.537281,0.000000,0.000000
13,High-corr filtered,LSTM_model_arch_24,0.697391,0.545390,0.520000,0.080620,0.690161,0.539474,1.000000,0.004739
8,High-corr filtered,LSTM_model_arch_19,0.702665,0.536170,0.480519,0.172093,0.690207,0.535088,0.400000,0.009479
12,High-corr filtered,LSTM_model_arch_23,0.697571,0.532624,0.464646,0.142636,0.691109,0.528509,0.166667,0.004739
6,High-corr filtered,LSTM_model_arch_17,0.696753,0.539007,0.333333,0.007752,0.691212,0.537281,0.000000,0.000000
5,High-corr filtered,LSTM_model_arch_16,0.679260,0.578723,0.545617,0.472868,0.691410,0.567982,0.570000,0.270142
15,High-corr filtered,LSTM_model_arch_26,0.697277,0.523404,0.451957,0.196899,0.691518,0.530702,0.200000,0.004739
16,High-corr filtered,LSTM_model_arch_27,0.694365,0.548936,0.534351,0.108527,0.693570,0.530702,0.466667,0.099526
0,High-corr filtered,LSTM_model_arch_11,0.685495,0.566667,0.527778,0.500775,0.695222,0.504386,0.461929,0.431280


In [65]:
# view model result dataset for XGBoost reduced data
best_models_xg_data

,Dataset,Model_Architecture,Loss,Accuracy,Precision,Recall,Validation_Loss,Validation_Accuracy,Validation_Precision,Validation_Recall
3,XGBoost,LSTM_model_arch_4,0.658053,0.613475,0.580906,0.556589,0.718918,0.506579,0.465686,0.450237
6,XGBoost,LSTM_model_arch_7,0.666759,0.606383,0.576792,0.524031,0.733486,0.497807,0.471519,0.706161
4,XGBoost,LSTM_model_arch_5,0.719356,0.579433,0.589655,0.265116,0.748150,0.491228,0.434783,0.331754
1,XGBoost,LSTM_model_arch_2,0.673665,0.583688,0.542398,0.575194,0.756880,0.502193,0.472222,0.644550
8,XGBoost,LSTM_model_arch_9,0.662511,0.602837,0.565891,0.565891,0.767336,0.467105,0.459596,0.862559
7,XGBoost,LSTM_model_arch_8,0.647282,0.621277,0.583710,0.600000,0.786677,0.473684,0.461126,0.815166
9,XGBoost,LSTM_model_arch_10,0.677223,0.556028,0.513828,0.547287,0.813225,0.489035,0.465625,0.706161
5,XGBoost,LSTM_model_arch_6,0.659748,0.620567,0.591362,0.551938,0.830245,0.475877,0.455128,0.672986
0,XGBoost,LSTM_model_arch_1,0.634198,0.634752,0.589532,0.663566,1.022237,0.506579,0.468750,0.497630
2,XGBoost,LSTM_model_arch_3,0.412617,0.788652,0.740638,0.827907,1.158542,0.480263,0.453901,0.606635
